In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile
import pandas as pd
from glob import glob
from os.path import join
import PIL 
import random
import cv2 as cv
import seaborn as sns

In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
DEFAULT_PATH = "/content/drive/MyDrive/MS-Upgrad/MildDemented/"
# DEFAULT_PATH = "/content/drive/MyDrive/MS-Upgrad/ModerateDemented/"

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
    DEFAULT_PATH, label_mode=None, image_size=(128, 128), batch_size=32
)
dataset = dataset.map(lambda x: x / 255.0)

In [ ]:
for x in dataset:
    plt.axis("off")
    plt.imshow((x.numpy() * 255).astype("int32")[0])
    break

In [ ]:
# discriminator model

discriminator = keras.Sequential(
    [
        keras.Input(shape=(128,128,3)),
        layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        layers.ZeroPadding2D(padding=((0,1),(0,1))),
        layers.BatchNormalization(momentum=0.8),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.25),
        layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.25),
        layers.Conv2D(256, kernel_size=3, strides=2, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.25),
        layers.Conv2D(512, kernel_size=3, strides=2, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ],
    name='discriminator'
)

discriminator.summary()

In [ ]:
# generator model

latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(4 * 4 * 512, activation='relu'),
        layers.Reshape((4, 4, 512)),
        layers.UpSampling2D(),
        layers.Conv2D(512, kernel_size=3, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation('relu'),
        layers.UpSampling2D(),
        layers.Conv2D(512, kernel_size=3, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation('relu'),
        layers.UpSampling2D(),
        layers.Conv2D(256, kernel_size=3, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation('relu'),
        layers.UpSampling2D(),
        layers.Conv2D(256, kernel_size=3, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation('relu'),
        layers.UpSampling2D(),
        layers.Conv2D(128, kernel_size=3, padding='same'),
        layers.BatchNormalization(momentum=0.8),
        layers.Activation('relu'),
        layers.Conv2D(3, kernel_size=3, padding='same'),
        layers.Activation('tanh')
    ],
    name="generator",
)

generator.summary()

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # fake images decoding
        generated_images = self.generator(random_latent_vectors)

        # Combined images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # discriminations
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated_img_%03d_%d.jpg" % (epoch, i))

In [ ]:
epochs = 2000

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=2, latent_dim=latent_dim)]
)

In [ ]:
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
tf.keras.utils.plot_model(discriminator,to_file='discriminator.png', show_shapes=True, dpi=64)

In [ ]:
data_folder = "/content/drive/MyDrive/MS-Upgrad"

## images GAN MildDemented
df_MildDemented_GAN = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "MildDementedGAN", "*"))),
    "y": 0,
    "class": "MildDemented"
})
shuffled_mild_gan = df_MildDemented_GAN.sample(frac=1)
testsize_mild_gan = int(0.2 * len(shuffled_mild_gan))
# Test
mild_test_gan = shuffled_mild_gan[:testsize_mild_gan]
# Train and validation
mild_trainval_gan = shuffled_mild_gan[testsize_mild_gan:]
trainsize_mild_gan = int(0.8 * len(mild_trainval_gan))
# Train
shuffled_mild_gan_train = mild_trainval_gan[:trainsize_mild_gan]
#Validation
shuffled_mild_gan_val = mild_trainval_gan[trainsize_mild_gan:]


## images GAN ModerateDemented
df_ModerateDemented_gan = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "ModerateDementedGAN", "*"))),
    "y": 0,
    "class": "ModerateDemented"
})
shuffled_moderate_gan = df_ModerateDemented_gan.sample(frac=1)
testsize_moderate_gan = int(0.2 * len(shuffled_moderate_gan))
# Test
moderate_test_gan = shuffled_moderate_gan[:testsize_moderate_gan]
# Train and validation
moderate_trainval_gan = shuffled_moderate_gan[testsize_moderate_gan:]
trainsize_moderate_gan = int(0.8 * len(moderate_trainval_gan))
# Train
shuffled_moderate_gan_train = moderate_trainval_gan[:trainsize_moderate_gan]
#Validation
shuffled_moderate_gan_val = moderate_trainval_gan[trainsize_moderate_gan:]


## images NonDemented
df_NonDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "Alzheimer_MRI_4_classes_dataset/NonDemented", "*"))),
    "y": 0,
    "class": "NonDementia"
})
shuffled_non = df_NonDemented_imgs.sample(frac=1)
testsize_non = int(0.2 * len(shuffled_non))
# Test
non_test = shuffled_non[:testsize_non]
# Train and validation
non_trainval = shuffled_non[testsize_non:]
trainsize_non = int(0.8 * len(non_trainval))
# Train
shuffled_non_train = non_trainval[:trainsize_non]
#Validation
shuffled_non_val = non_trainval[trainsize_non:]


## images VeryMildDemented
df_VeryMildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "Alzheimer_MRI_4_classes_dataset/VeryMildDemented", "*"))),
    "y": 1,
    "class": "VeryMildDementia"
})
shuffled_verymild = df_VeryMildDemented_imgs.sample(frac=1)
testsize_verymild = int(0.2 * len(shuffled_verymild))

# Test
verymild_test = shuffled_verymild[:testsize_verymild]

# Train and validation
verymild_trainval = shuffled_verymild[testsize_verymild:]
trainsize_verymild = int(0.8 * len(verymild_trainval))

# Train
shuffled_verymild_train = verymild_trainval[:trainsize_verymild]

#Validation
shuffled_verymild_val = verymild_trainval[trainsize_verymild:]


## images MildDemented
df_MildDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "Alzheimer_MRI_4_classes_dataset/MildDemented", "*"))),
    "y": 2,
    "class": "MildDementia"
})
shuffled_mild = df_MildDemented_imgs.sample(frac=1)
testsize_mild = int(0.2 * len(shuffled_mild))

# Test
mild_test = shuffled_mild[:testsize_mild]

# Train and validation
mild_trainval = shuffled_mild[testsize_mild:]
trainsize_mild = int(0.8 * len(mild_trainval))

# Train
shuffled_mild_train = mild_trainval[:trainsize_mild]

#Validation
shuffled_mild_val = mild_trainval[trainsize_mild:]

## images ModerateDemented
df_ModerateDemented_imgs = pd.DataFrame({
    "X": sorted(glob(join(data_folder, "Alzheimer_MRI_4_classes_dataset/ModerateDemented", "*"))),
    "y": 3,
    "class": "ModerateDementia"
})
shuffled_moderate = df_ModerateDemented_imgs.sample(frac=1)
testsize_moderate = int(0.2 * len(shuffled_moderate))

# Test
moderate_test = shuffled_moderate[:testsize_moderate]

# Train and validation
moderate_trainval = shuffled_moderate[testsize_moderate:]
trainsize_moderate = int(0.8 * len(moderate_trainval))

# Train
shuffled_moderate_train = moderate_trainval[:trainsize_moderate]

#Validation
shuffled_moderate_val = moderate_trainval[trainsize_moderate:]


## Number of images
print("TOTAL:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate))
print("# of images with MildDemented Alzheimer GAN =", len(shuffled_mild_gan))
print("# of images with ModerateDemented Alzheimer GAN =", len(shuffled_moderate_gan))

print("------------")
print("\nTest:")
print("# of images with NonDemented Alzheimer =", len(non_test))
print("# of images with VeryMildDemented Alzheimer =", len(verymild_test))
print("# of images with MildDemented Alzheimer =", len(mild_test))
print("# of images with ModerateDemented Alzheimer =", len(moderate_test))
print("# of images with MildDemented Alzheimer GAN =", len(mild_test_gan))
print("# of images with ModerateDemented Alzheimer GAN =", len(moderate_test_gan))
print("------------")
print("\nTraining:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non_train))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild_train))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild_train))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate_train))
print("# of images with MildDemented Alzheimer GAN=", len(shuffled_mild_gan_train))
print("# of images with ModerateDemented Alzheimer GAN =", len(shuffled_moderate_gan_train))
print("------------")
print("\nValidation:")
print("# of images with NonDemented Alzheimer =", len(shuffled_non_val))
print("# of images with VeryMildDemented Alzheimer =", len(shuffled_verymild_val))
print("# of images with MildDemented Alzheimer =", len(shuffled_mild_val))
print("# of images with ModerateDemented Alzheimer =", len(shuffled_moderate_val))
print("# of images with MildDemented Alzheimer gan=", len(shuffled_mild_gan_val))
print("# of images with ModerateDemented Alzheimer gan=", len(shuffled_moderate_gan_val))


In [ ]:
heights = [len(shuffled_non), len(shuffled_verymild), (len(shuffled_mild) + len(shuffled_mild_gan)), (len(shuffled_moderate) + len(shuffled_moderate_gan))]

fig, ax = plt.subplots()
height = heights
bars = ('Non', 'Very Mild', 'Mild', 'Moderate')
y_pos = np.arange(len(bars))
plt.bar(y_pos, height)
plt.xticks(y_pos, bars)
plt.show()

In [ ]:
def load_image(fname):
    """
    Load an image using opencv given its path.
    """
    # img is a numpy array
    img = cv.imread(fname)
    # opencv uses BGR channel order by default
    # so convert to RGB
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    # resize image
    resize = (224, 224)
    img = cv.resize(img, resize)
    return img

def hot_array(class_number):
    ha = [0,0,0,0]
    ha[class_number] = 1
    return ha

In [ ]:
# plot some images per class

ncols = 10

fig, axs = plt.subplots(nrows=4, ncols=ncols, figsize=(20, 10))

for fname,ax in zip(shuffled_mild_gan.loc[:ncols, "X"], axs[0,:]):
    im = load_image(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

for fname,ax in zip(shuffled_mild.loc[:ncols, "X"], axs[2,:]):
    im = load_image(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

for fname,ax in zip(shuffled_moderate_gan.loc[:ncols, "X"], axs[1,:]):
    im = load_image(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)


for fname,ax in zip(shuffled_moderate.loc[:ncols, "X"], axs[3,:]):
    im = load_image(fname)
    ax.imshow(im)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)

axs[0,0].set_ylabel("Mild Dementia - GAN")
axs[1,0].set_ylabel("Mild Dementia")
axs[2,0].set_ylabel("Moderate Dementia - GAN")
axs[3,0].set_ylabel("Moderate Dementia")

In [ ]:
from psutil import *

In [ ]:
cpu_count()

In [ ]:
cpu_stats()

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!df -h

In [ ]:
virtual_memory()

In [ ]:
!nvidia-smi -L

In [ ]:
!lscpu | grep "L3 cache" 